In [103]:
from pandas import read_table
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from pandas import DataFrame
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import datetime as dt
import os
import shutil
from pandas import read_excel
from pandas import concat
from pandas import merge
import pandas as pd
import numpy as np


from matplotlib import pyplot as plt

In [104]:
chromedriver_autoinstaller.install()

'c:\\users\\junhong\\appdata\\local\\programs\\python\\python38\\lib\\site-packages\\chromedriver_autoinstaller\\91\\chromedriver.exe'

In [105]:
# 크롬드라이버 실행
driver = webdriver.Chrome()

# 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
driver.implicitly_wait(5)

In [106]:
driver.get('https://data.seoul.go.kr/dataList/104/S/2/datasetView.do')
time.sleep(3)


In [107]:
html = driver.page_source

In [ ]:
year_list = np.arange(2002,2020)
txt_list = []

#대기정보df = DataFrame()
    
for i in year_list:
    select = Select(driver.find_element_by_css_selector('select[id="statStartPeriod"]'))
    select.select_by_value('%s' %str(i))
    
    select = Select(driver.find_element_by_css_selector('select[id="statEndPeriod"]'))
    select.select_by_value('%s' %str(i))
    
    click_button = WebDriverWait(driver, 3).until(
                    lambda x: x.find_element_by_css_selector("#searchBtn"))
    click_button.click()
    time.sleep(10)

    click_button = WebDriverWait(driver, 3).until(
                    lambda x: x.find_element_by_css_selector("#btnCsv"))
    click_button.click()
    try:
        alert = Alert(driver)
        alert.accept()
    except:
        'No Alert'
    time.sleep(10)
    
    txt_list.append('%s_사업체수.txt' %str(i))
    shutil.copy('C:/Users/junhong/Downloads/report.txt', '%s_사업체수.txt' %str(i))
    os.remove('C:/Users/junhong/Downloads/report.txt')
    
    #xlsx = read_excel('sidoAirInfo_%s.xls' %str(i), skiprows = 3)
    #tmp_df = DataFrame(xlsx)
    #대기정보df = concat([대기정보df, tmp_df])

In [133]:
df_2002_2005 = DataFrame()

for item in txt_list[:4]:
    txt = read_table('%s' %item,sep = '\t')
    txt = txt.drop([0,1,2])
    for i in txt:
            if i.find('.') != -1:
                txt = txt.drop([i],axis = 1)
    txt = txt.set_index(['기간','자치구'])

    for i in txt:
        txt[i] = txt[i].str.replace(',','')

    txt = txt.apply(pd.to_numeric, errors = 'coerce')

    check_na = txt.isna().sum()
    na_columns = []
    for i in check_na.keys():
        if check_na[i] != 0:
            na_columns.append(i)

    for i in na_columns:
        txt[i].fillna(0, inplace = True)

    df_2002_2005 = concat([df_2002_2005, txt])
    #os.remove('%s' %item)
df_2002_2005

합계  농업및임업   어업    광업    제조업  전기가스수도사업   건설업   도소매업  숙박및음식점업  \
기간   자치구                                                                   
2002 종로구  39073    1.0  2.0   5.0   4718         2   355  17676     6336   
     중구   68942    0.0  1.0   3.0  11480         5   418  38756     6071   
     용산구  21985    0.0  0.0   0.0   1807         4   406   9262     3759   
     성동구  24264    0.0  0.0   0.0   5456         6   416   5912     3802   
     광진구  24613    3.0  0.0   0.0   2606         7   614   7172     4596   
...         ...    ...  ...   ...    ...       ...   ...    ...      ...   
2005 관악구  26480    1.0  0.0   0.0   1676         4   779   5723     5618   
     서초구  36347    3.0  2.0   6.0   1894        13  2154  11109     5410   
     강남구  53667    3.0  4.0  11.0   2505        27  2523  13307     8681   
     송파구  40400    2.0  2.0   7.0   1972         8  1830  12644     5885   
     강동구  27052    0.0  0.0   2.0   1621        10   890   6279     4436   

           운수업  통신업  금융및보험업  부동산업및임대업  사업서비스업  공공행정국방및사회보장행정  교육서비스업  \
기간   자치구                                                               
2002 종로구  1708  100     599       964    1603            106     602   
     중구   2274  203    1127      1079    3459             66     325   
     용산구  1705   84     212       840     626             47     409   
     성동구  2829   63     410       927     486             49     596   
     광진구  2534   56     188      1037     828             45     878   
...        ...  ...     ...       ...     ...            ...     ...   
2005 관악구  4093   61     184      1326     656             60    1085   
     서초구  1337  151     537      2208    5277             71    1135   
     강남구  3204  258    1238      4029    7781             70    1880   
     송파구  5330  106     417      2187    1851             72    1566   
     강동구  5524   49     249      1451     541             39    1127   

          보건및사회복지사업  오락문화및운동관련서비스업  기타공공및개인서비스업  
기간   자치구                                         
2002 종로구        597           1151         2548  
     중구         528            924         2223  
     용산구        324            603         1897  
     성동구        418            746         2148  
     광진구        535           1153         2361  
...             ...            ...          ...  
2005 관악구        689           1507         3018  
     서초구        921           1291         2828  
     강남구       1898           2217         4031  
     송파구       1024           1804         3693  
     강동구        756           1263         2815  

[100 rows x 19 columns]

In [140]:
df = DataFrame()

for item in txt_list[13:]:
    txt = read_table('%s' %item,sep = '\t')

    txt = txt.drop([0,1,2])
    for i in txt:
        if i.find('.') != -1:
            txt = txt.drop([i],axis = 1)
    txt = txt.query('동 == "소계"').drop(['동'], axis = 1).set_index(['기간','자치구'])

    for i in txt:
        txt[i] = txt[i].str.replace(',','')

    txt = txt.apply(pd.to_numeric, errors = 'coerce')

    check_na = txt.isna().sum()
    na_columns = []
    for i in check_na.keys():
        if check_na[i] != 0:
            na_columns.append(i)

    for i in na_columns:
        txt[i].fillna(0, inplace = True)
    
    df = concat([df, txt])
    #os.remove('%s' %item)

df

합계  농업 임업 및 어업   광업    제조업  전기 가스 증기 및 수도사업  \
기간   자치구                                                   
2015 종로구  40871         1.0  2.0   4281              6.0   
     중구   66190         0.0  3.0  10259              7.0   
     용산구  21178         0.0  0.0   1100              3.0   
     성동구  26130         0.0  0.0   4650             10.0   
     광진구  24531         0.0  0.0   1917              2.0   
...         ...         ...  ...    ...              ...   
2019 관악구  25505         0.0  0.0   1185              NaN   
     서초구  46940         3.0  3.0   1068              NaN   
     강남구  71027         5.0  3.0   1650              NaN   
     송파구  48644         6.0  1.0   1534              NaN   
     강동구  29080         0.0  0.0   1292              NaN   

          하수 · 폐기물 처리 원료재생 및 환경복원업   건설업  도매 및 소매업     운수업  숙박 및 음식점업  ...  \
기간   자치구                                                               ...   
2015 종로구                       9.0   358     17447  1418.0       6635  ...   
     중구                        8.0   451     34785  1666.0       6142  ...   
     용산구                      12.0   440      7714  1292.0       4245  ...   
     성동구                      31.0   661      7090  2641.0       3494  ...   
     광진구                      16.0   836      6214  2511.0       4555  ...   
...                            ...   ...       ...     ...        ...  ...   
2019 관악구                       NaN   709      5047     NaN       5662  ...   
     서초구                       NaN  1603     11890     NaN       6554  ...   
     강남구                       NaN  1809     16401     NaN      11037  ...   
     송파구                       NaN  1851     13699     NaN       7461  ...   
     강동구                       NaN   989      6425     NaN       4879  ...   

          보건업 및 사회복지 서비스업  예술 스포츠 및 여가관련 서비스업  협회 및 단체 수리  및 기타 개인 서비스업  \
기간   자치구                                                                  
2015 종로구              689                 927                    2592.0   
     중구               697                 584                    2408.0   
     용산구              541                 397                    1722.0   
     성동구              654                 522                    2066.0   
     광진구              849                 875                    2618.0   
...                   ...                 ...                       ...   
2019 관악구             1179                1061                       NaN   
     서초구             1663                1343                       NaN   
     강남구             3042                1840                       NaN   
     송파구             1876                1726                       NaN   
     강동구             1277                 930                       NaN   

          전기 가스 증기 및 공기조절 공급업  수도 하수 및 폐기물 처리 원료 재생업  운수 및 창고업   정보통신업  \
기간   자치구                                                                 
2015 종로구                  NaN                    NaN       NaN     NaN   
     중구                   NaN                    NaN       NaN     NaN   
     용산구                  NaN                    NaN       NaN     NaN   
     성동구                  NaN                    NaN       NaN     NaN   
     광진구                  NaN                    NaN       NaN     NaN   
...                       ...                    ...       ...     ...   
2019 관악구                  1.0                    9.0    3414.0   265.0   
     서초구                 18.0                   18.0    1611.0  2286.0   
     강남구                 58.0                   22.0    3193.0  4199.0   
     송파구                 10.0                   29.0    5836.0   939.0   
     강동구                  1.0                   17.0    5417.0   204.0   

            부동산업  사업시설 관리 사업 지원 및 임대 서비스업  협회 및 단체 수리 및 기타 개인 서비스업  
기간   자치구                                                            
2015 종로구     NaN                      NaN                      NaN  
     중구      NaN                      NaN                      NaN  
  